In [183]:
import numpy as np
import pandas as pd

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)

import datetime
from datetime import datetime, date

#### Загрузка сырья из файла
* [x] Проблема с ключом. Не во всех строках есть ключ. Удаляем их. Потом создаём ключ.
* [x] Столбцы Unnamed и Строка1 пустые. Удалить.
* [ ] По уму надо удалять все колонки, где ничего нет. Но пока указываем вручную.
* [x] Распознать даты.

In [185]:
history_file = '/Users/op/dev/olgapavlova/agile-health-hackathon/csv/history-Table 1.csv'

date_parser = lambda x: pd.to_datetime(x, format='%d/%m/%Y HH:MM')

dhis = pd.read_csv(history_file,
                   dtype = {
                               'entity_id': 'Int64',
                               'history_version': 'Int64',
                   },
                   header=1,               # в первой строчке мусор, во второй — названия колонок
                   delimiter=';')

##### Установка ключа

In [187]:
dhis = dhis.dropna(subset=['entity_id'])
dhis.set_index('entity_id')
dhis.dtypes

entity_id                  Int64
history_property_name     object
history_date              object
history_version            Int64
history_change_type       object
history_change            object
Столбец1                 float64
Unnamed: 7               float64
dtype: object

##### Удаление заведомо пустых колонок

In [194]:
dhis = dhis.drop(columns=['Столбец1', 'Unnamed: 7'])
dhis.dtypes

entity_id                         Int64
history_property_name            object
history_date             datetime64[ns]
history_version                   Int64
history_change_type              object
history_change                   object
dtype: object

#### Номер версии?

In [80]:
dhis.groupby('history_version').size()

history_version
1      4904
2      4612
3      3453
4      2762
5      2856
       ... 
115       1
116       1
117       1
118       1
119       1
Length: 119, dtype: int64

# название_свойства; лог_изменения_свойства; дата_и_время

In [200]:
dhis.groupby('history_property_name').size()

history_property_name
Story Points                   1
Владелец                     490
Вложения                     498
Время Исправления (ПЛАН)     210
Время Исправления (ФАКТ)      60
                            ... 
Тип дефекта                    8
Тип работ                     30
Учет рабочего времени       1639
Фаза внесения дефекта          2
Фаза обнаружения дефекта      11
Length: 68, dtype: int64

##### Преобразование с заменой некорректных дат на NaT

In [189]:
dhis['history_date'] = pd.to_datetime(dhis['history_date'], format='%m/%d/%y %H:%M', errors='coerce')
dhis['history_date']

0       2024-09-10 11:17:00
1       2024-09-10 11:17:00
2       2023-07-13 11:07:00
4       2023-07-21 11:06:00
5       2023-07-31 15:05:00
                ...        
64174   2024-10-23 07:00:00
64175   2024-10-23 07:00:00
64176   2024-10-23 07:00:00
64177   2024-10-23 07:00:00
64178   2024-10-23 07:00:00
Name: history_date, Length: 61120, dtype: datetime64[ns]

#### Изменяемое свойство
* Иногда это колонка из основного среза данных. Но не всегда.
* Нас интересуют многие из этих колонок. Но не все.
* Возможно, наборы названий там и здесь расходятся.

#### Тип изменений?
* Есть пустые, но немного.

In [219]:
dhis['history_change_type'].isna().sum()

653

In [202]:
dhis.groupby('history_change_type').size()

history_change_type
CREATED           2481
FIELD_CHANGED    57986
dtype: int64

#### История изменений
* Что на что менялось в колонке, указанной в history_property_name.
* В некоторых строках колонки не указаны.

In [239]:
pd.set_option('display.max_colwidth', None)
d2 = dhis[['history_property_name', 'history_change']]
d2 = d2[d2['history_property_name'] == 'Статус']
d2

      history_property_name         history_change
8                    Статус    created -> analysis
9                    Статус     analysis -> fixing
10                   Статус      fixing -> testing
12                   Статус      testing -> closed
64                   Статус    created -> analysis
...                     ...                    ...
64147                Статус  created -> inProgress
64152                Статус     inProgress -> done
64153                Статус         done -> closed
64167                Статус  created -> inProgress
64178                Статус   inProgress -> closed

[6964 rows x 2 columns]

In [214]:
dhis['history_change'].isna().sum()

7150